In [1]:
import traceviz.client
import numpy as np
from traceviz.proto import  viz_pb2
import json
# ---
from bayes3d._mkl.utils import keysplit
import os
import matplotlib.pyplot as plt
import numpy as np
import jax
from jax import jit, vmap
import jax.numpy as jnp

key = jax.random.PRNGKey(0)

## Spheres

In [119]:
t = 100
n = 100
key, keys = keysplit(key, 1, 3)
x0 = jax.random.normal(keys[0], (1,n,3))
xs = x0 + jnp.cumsum(0.03*jax.random.uniform(keys[0], (t,n,3)), axis=0)
cs = jnp.tile( jax.random.uniform(keys[1], (n,4)), (t,1,1))
ss = 0.1*jnp.tile( jax.random.uniform(keys[2], (n,4)), (t,1,1))

xs.shape, cs.shape

((100, 100, 3), (100, 100, 4))

In [120]:
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "setup"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg([]))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "animated spheres"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
        'centers': np.array(xs), 
        'colors':  np.array(cs),
        'scales':  np.array(ss)
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))

response:  listener_identifiers: "2023-11-30T16:28:31.217097 :: ipv4:127.0.0.1:36214"

response:  listener_identifiers: "2023-11-30T16:28:31.217097 :: ipv4:127.0.0.1:36214"



## I Phone video data

In [157]:
import bayes3d


data = np.load("data/ramen.npz")
depths = data["depths"]
colors= data["colors"]

intrinsics = bayes3d.Intrinsics(*data["intrinsics"])

xs = np.array([bayes3d.transforms_3d.unproject_depth(depths[i], intrinsics) for i in range(len(colors))])
cs = np.array([bayes3d.utils.resize(colors[i], int(intrinsics.height), int(intrinsics.width)) for i in range(len(colors))])
cs = np.concatenate([cs, np.ones((*cs.shape[:3], 1))], axis=-1)/255
ss = np.ones(xs.shape[:3] + (1,))

xs[:, :, :, 1] *= -1

T = xs.shape[0]
N = xs.shape[1]*xs.shape[2] 
T, N, depths.shape, colors.shape, xs.shape, cs.shape

(56,
 49152,
 (56, 256, 192),
 (56, 960, 720, 3),
 (56, 256, 192, 3),
 (56, 256, 192, 4))

In [153]:
xs = xs - np.mean(xs, axis=(0,1,2))

In [160]:
k = -1
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "setup"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg([]))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "spheres"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
        'centers': xs.reshape(T,N,3)[0,:k], 
        'colors':  cs.reshape(T,N,4)[0,:k], 
        'scales':  0.001*ss.reshape(T,N,1)[0,:k], 
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))

response:  listener_identifiers: "2023-11-30T16:54:34.639417 :: ipv4:127.0.0.1:57890"

response:  listener_identifiers: "2023-11-30T16:54:34.639417 :: ipv4:127.0.0.1:57890"



In [161]:
k = -1
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "setup"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg([]))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "animated spheres"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
        'centers': xs.reshape(T,N,3)[:,:k], 
        'colors':  cs.reshape(T,N,4)[:,:k], 
        'scales':  0.001*ss.reshape(T,N,1)[:,:k], 
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))

response:  listener_identifiers: "2023-11-30T16:54:34.639417 :: ipv4:127.0.0.1:57890"

response:  listener_identifiers: "2023-11-30T16:54:34.639417 :: ipv4:127.0.0.1:57890"



In [ ]:
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "setup"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
    "test": "I am a test string"
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "animated gaussians"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
        'transforms': np.array(transforms[:]), 
        'colors':  np.array(cs[:])
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))

In [53]:
from bayes3d._mkl.trimesh_to_gaussians import pack_transform, ellipsoid_embedding

t = 100
n = 100
key, keys = keysplit(key, 1, 3)
x0 = jax.random.normal(keys[0], (1,n,3))
xs = x0 + jnp.cumsum(0.03*jax.random.uniform(keys[0], (t,n,3)), axis=0)
cs = jnp.tile( jax.random.uniform(keys[1], (n,4)), (t,1,1))
ss = 0.1*jnp.tile( jax.random.uniform(keys[2], (n,4)), (t,1,1))


choleskys = jax.random.normal(key, (n,3,3))
choleskys = jnp.tile(choleskys, (t,1,1,1))
transforms = vmap(pack_transform, (0,0,None))(xs.reshape(-1,3), choleskys.reshape(-1,3,3), 0.26)
transforms = transforms.reshape(t,n,4,4)
transforms.shape, choleskys.shape

((100, 100, 4, 4), (100, 100, 3, 3))

In [58]:
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "setup"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
    "test": "I am a test string"
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))
msg = viz_pb2.Message()
msg.payload.json = json.dumps({"type": "animated gaussians"})
msg.payload.data.MergeFrom(traceviz.client.to_pytree_msg({
        'transforms': np.array(transforms[:]), 
        'colors':  np.array(cs[:])
}))
stub = traceviz.client.connect()
print('response: ', stub.Broadcast(msg))

ValueError: Unhandled pytree input type: <class 'str'>. Value:
I am a test string